http://www.tensorfly.cn/tfdoc/tutorials/mnist_beginners.html

# 实现回归模型

In [1]:
import tensorflow as tf

x不是一个特定的值，而是一个**占位符placeholder**，我们在TensorFlow运行计算时输入这个值。

In [2]:
x = tf.placeholder("float", [None, 784])

**Variable**代表可修改的张量，在TensorFlow用于描述交互性操作的图中。

可以用于计算输入值，也可以在计算中被修改。对于各种机器学习应用，一般都会有模型参数，可以用Variable表示。

tf.zeros：创建一个所有的参数为0的tensor对象

In [3]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

tf.matmul(X，W)表示x乘以W。加上b，把和输入到tf.nn.softmax（）函数里面。

In [4]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

# 训练模型

为了计算交叉熵，我们首先需要添加一个新的占位符用于输入正确值：

In [5]:
y_ = tf.placeholder("float", [None,10])

### 计算交叉熵（主要用于度量两个概率分布间的差异性信息）

tf.log 计算 y 的每个元素的对数。

接下来，我们把 y_ 的每一个元素和 tf.log(y_) 的对应元素相乘。

最后，用 tf.reduce_sum（） 计算张量的所有元素的总和。

reduce_sum()按照求和的方式对矩阵降维。reduce_mean()按照某个维度求平均值。

In [6]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

TensorFlow用梯度下降算法（gradient descent algorithm）以0.01的学习速率，最小化交叉熵。

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

添加一个操作来初始化我们创建的变量：

In [8]:
init = tf.global_variables_initializer()

在一个Session里面启动我们的模型，并且初始化变量：

In [9]:
sess = tf.Session()
sess.run(init)

### 导入数据集

In [15]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


训练模型，这里我们让模型循环训练1000次

In [11]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# 评估模型

### tf.argmax（）给出某个tensor对象在某一维上的其数据最大值所在的索引值。
标签向量由0,1组成，因此最大值1所在的索引位置就是类别标签。

比如tf.argmax(y,1)返回的是模型对于任一输入x预测到的标签值，而 tf.argmax(y_,1) 代表正确的标签，

我们可以用 tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)。

In [12]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

为了确定正确预测项的比例 tf.cast 类型转换 把布尔值转换成浮点数 ，然后取平均值

例如，[True, False, True, True] 会变成 [1,0,1,1] ，取平均值后得到 0.75.

In [13]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

计算所学习到的模型在测试数据集上面的正确率

In [14]:
print (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9175
